# UE boss damage simulation
This attempts to simulate hitting bosses with certain lt configurations.

For now it only supports Overkill and TheCollector as well as any regular dmg, crit and crit dmg lts.

This does *not* support sks for now. It also doesn't consider lt crits since we don't know the proc chance for that and we assume a baseline of crits dealing double damage without any further investment into crit dmg.

In [25]:
BASE_CRIT_DAMAGE = 100 # this means crit deal double dmg by default

In [26]:
import random
import pandas as pd
from typing import List, Tuple, Dict

BASE_CRIT_DAMAGE = 50  # Adding a sensible base critical damage value

def simulate_damage(
    base_damage: float = 10.0,
    percent_damage_mod: float = 0.0,
    crit_chance: float = 25.0,
    crit_damage: float = 50.0,
    num_hits: int = 10000,
    use_ok: bool = True,
    ok_proc_chance: float = 10.0,
    collector_multiplier: float = 3.0,
    use_tanya: bool = False,
    tanya_dmg: int = 50,
) -> Tuple[List[float], List[Dict[str, float]]]:
    """
    Simulate damage for a series of hits, with optional critical hits,
    overkill procs, and collector procs.
    """
    hits = []
    proc_data = []

    for _ in range(num_hits):
        # Calculate the regular damage with modifiers
        regular_damage = base_damage * (1 + percent_damage_mod / 100)

        is_crit = False
        collector_proc_occurred = False
        tanya_proc_occurred = False
        proc_occurred = False
        proc_depth = 0

        # Determine if the hit is a critical hit
        if random.random() < crit_chance / 100:
            is_crit = True
            crit_extra_damage = regular_damage * ((crit_damage + BASE_CRIT_DAMAGE) / 100)
            damage = regular_damage + crit_extra_damage
            
            if use_ok:
                # Handle Overkill chain calculation
                multiplier = 1
                while random.random() < ok_proc_chance / 100:
                    proc_occurred = True
                    proc_depth += 1
                    multiplier *= 2
                    if multiplier >= 1000:
                        break
                crit_extra_damage *= multiplier
                damage = regular_damage + crit_extra_damage

            if use_tanya:
                if random.random() < 30 / 100:
                    tanya_proc_occurred = True
                    damage *= 1 + tanya_dmg / 100

            # Handle the collector proc (only on crit)
            if random.random() < 15 / 100:
                collector_proc_occurred = True
                damage *= collector_multiplier
        else:
            damage = regular_damage

        hits.append(damage)
        # Build the proc_data dictionary conditionally
        proc_entry = {
            'Damage': damage,
            'IsCrit': is_crit,
            'CollectorProcOccurred': collector_proc_occurred
        }
        
        if use_ok:
            proc_entry['OKProcOccurred'] = proc_occurred
            proc_entry['OKProcDepth'] = proc_depth
        
        if use_tanya:
            proc_entry['TanyaProcOccurred'] = tanya_proc_occurred

        proc_data.append(proc_entry)

    return hits, proc_data


## With Overkill and Collector

In [27]:
# Example usage
num_hits = 100000
base_damage = 10
crit_chance = 56  # 25% chance to crit
crit_damage = 155  # 50% additional damage on crit
use_ok = True
ok_proc_chance = 24  # 10% chance to proc on crit
percent_damage_mod = 117  # 20% additional damage
collector_multiplier = 3.5  # Multiplier for the entire hit
use_tanya = False
tanya_dmg = 90

hits, proc_data = simulate_damage(
    base_damage=base_damage, 
    percent_damage_mod=percent_damage_mod, 
    crit_chance=crit_chance, 
    crit_damage=crit_damage, 
    num_hits=num_hits, 
    use_ok=use_ok, 
    ok_proc_chance=ok_proc_chance, 
    collector_multiplier=collector_multiplier,
    use_tanya=use_tanya,
    tanya_dmg=tanya_dmg)

# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

# Basic statistics
print(df_hits.describe())
print("\n")
print(df_proc_data.value_counts())

              Damage
count  100000.000000
mean       76.836564
std       146.830063
min        21.700000
25%        21.700000
50%        66.185000
75%        66.185000
max     22798.020000


Damage      IsCrit  CollectorProcOccurred  OKProcOccurred  OKProcDepth
21.7000     False   False                  False           0              44040
66.1850     True    False                  False           0              36158
110.6700    True    False                  True            1               8585
231.6475    True    True                   False           0               6332
199.6400    True    False                  True            2               2087
387.3450    True    True                   True            1               1639
377.5800    True    False                  True            3                515
698.7400    True    True                   True            2                373
733.4600    True    False                  True            4                112
1321.5300   True  

## With Tanya and Collector

In [28]:
# Example usage
num_hits = 100000
base_damage = 10
crit_chance = 48  # 25% chance to crit
crit_damage = 155  # 50% additional damage on crit
use_ok = False
ok_proc_chance = 24  # 10% chance to proc on crit
percent_damage_mod = 117  # 20% additional damage
collector_multiplier = 3.5  # Multiplier for the entire hit
use_tanya = True
tanya_dmg = 90

hits, proc_data = simulate_damage(
    base_damage=base_damage, 
    percent_damage_mod=percent_damage_mod, 
    crit_chance=crit_chance, 
    crit_damage=crit_damage, 
    num_hits=num_hits, 
    use_ok=use_ok, 
    ok_proc_chance=ok_proc_chance, 
    collector_multiplier=collector_multiplier,
    use_tanya=use_tanya,
    tanya_dmg=tanya_dmg)

# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

# Basic statistics
print(df_hits.describe())
print("\n")
print(df_proc_data.value_counts())

              Damage
count  100000.000000
mean       66.471990
std        76.145097
min        21.700000
25%        21.700000
50%        21.700000
75%        66.185000
max       440.130250


Damage     IsCrit  CollectorProcOccurred  TanyaProcOccurred
21.70000   False   False                  False                52170
66.18500   True    False                  False                28456
125.75150  True    False                  True                 12255
231.64750  True    True                   False                 5001
440.13025  True    True                   True                  2118
Name: count, dtype: int64
